# Convolution Neural Network(CNN) to Classify Human's Feelings

## IMPORTANT: This Project is 100% done by Max, HONG Ka Ho serving as academic purpose only

Recently, I stumple upon a very interesting data set at Kaggle which is a data set of 275 bird species with 39364 training images, 1375 test images(5 per species) and 1375 validation images. To make things simplier, I would only consider 15 species only. I think it would be a great place for me to apply my knowledge of CNN to see if I can successfully build a trust-worth model to indicate the species that that specific bird belongs to. I can't wait to share my project results to you.

Source of the dataset: https://www.kaggle.com/gpiosenka/100-bird-species

# Step 1: Data Preprocessing

![pcscreenshot](image3.png "File List")
![pcscreenshot](image1.png "File List")
![pcscreenshot](image2.png "File List")

We can see from the above images that all the files are being arranged regularly that are segregated into train, test and validation set which each file inside comprised 275 files, one specy one file

In [2]:
# Import Relevant Library
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
from tensorflow.keras.preprocessing.image import load_img
import sys
from PIL import Image
import os
import numpy as np
from datetime import datetime
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
sys.modules['Image'] = Image

### Preprocessing the Training Set

In [3]:
# we would apply transformation of image to avoid over-fitting problems
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
training_set = train_datagen.flow_from_directory(
        'archive1/train',
        target_size = (224, 224),
        batch_size = 32,
        class_mode = "categorical")


Found 2202 images belonging to 15 classes.


### Preprocessing the Validation Set and Test Set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'archive1/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
valid_datagen = ImageDataGenerator(rescale=1./255)
valid_set = test_datagen.flow_from_directory(
        'archive1/valid',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 75 images belonging to 15 classes.
Found 75 images belonging to 15 classes.


# Build the CNN Model

In [5]:
#initialize cnn
cnn = tf.keras.models.Sequential()
# add convolution layer to cnn
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu',input_shape = (224, 224, 3), padding="valid"))# Add Pooling to convolution layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2))
# Add additional convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2))
# flattening the convolution layers
cnn.add(tf.keras.layers.Flatten())
# Fully connect the layers
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
# create output layer
cnn.add(tf.keras.layers.Dense(units = 15, activation = 'softmax'))
# compile CNN
cnn.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'])
# activate early stopping to avoid overfitting
es = EarlyStopping(monitor = 'loss', min_delta = 1e-10, patience = 10, verbose = 1)
# reduce learning rate when learning plateau
rlr = ReduceLROnPlateau(monitor = 'loss', factor = 0.2, patience = 5, verbose = 1)
tb = TensorBoard(log_dir = logdir)
# train the cnn
cnn.fit(x = training_set, validation_data = valid_set, epochs = 30, callbacks = [rlr, es, tb])

Epoch 1/30
69/69 [==============================] - 44s 573ms/step - loss: 2.7312 - accuracy: 0.2357 - val_loss: 1.7812 - val_accuracy: 0.3733
Epoch 2/30
69/69 [==============================] - 37s 534ms/step - loss: 1.5041 - accuracy: 0.5282 - val_loss: 1.0642 - val_accuracy: 0.6133
Epoch 3/30
69/69 [==============================] - 37s 532ms/step - loss: 1.0833 - accuracy: 0.6599 - val_loss: 0.6085 - val_accuracy: 0.7600
Epoch 4/30
69/69 [==============================] - 37s 530ms/step - loss: 0.9068 - accuracy: 0.7048 - val_loss: 0.6092 - val_accuracy: 0.8000
Epoch 5/30
69/69 [==============================] - 37s 536ms/step - loss: 0.8227 - accuracy: 0.7293 - val_loss: 0.4153 - val_accuracy: 0.8533
Epoch 6/30
69/69 [==============================] - 36s 520ms/step - loss: 0.6705 - accuracy: 0.7861 - val_loss: 0.4568 - val_accuracy: 0.8000
Epoch 7/30
69/69 [==============================] - 37s 533ms/step - loss: 0.5433 - accuracy: 0.8297 - val_loss: 0.3263 - val_accuracy: 0.9067

Result of the CNN after 30 epochs has reached around 99% accuracy in training set and 92% accuracy in validation set respectively which are pretty good results! Let's try to apply our model to the test set

In [6]:
# save the model
cnn.save('save/', save_format = 'tf')

INFO:tensorflow:Assets written to: save/assets


## Testing of Model

In this session, we are going to choose one picture from each category and then pass them into the model to check their prediction accuracy. 
![pcscreenshot](image5.png "File List")


In [9]:
from tensorflow.keras.preprocessing import image
species_name = os.listdir('archive1/train')
predicted_set = []
m = []
for bird_specy in species_name:
    for i in range(1, 6):
        test_image = image.load_img('archive1/test/{}/{}.jpg'.format(bird_specy, i), target_size = (224, 224))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = cnn.predict(test_image / 255.0)
        max_index = np.where((result == np.amax(result))[0])[0][0]
        predicted_species = species_name[max_index]
        predicted_set.append(predicted_species)
        if predicted_species == bird_specy:
            m.append(1)
        else:
            m.append(0)
accuracy = np.sum(m) / len(m)
print(accuracy)

0.8933333333333333


In [10]:
# look at the predictions
predicted_set

['CALIFORNIA CONDOR',
 'AFRICAN CROWNED CRANE',
 'AFRICAN CROWNED CRANE',
 'AFRICAN CROWNED CRANE',
 'AFRICAN CROWNED CRANE',
 'AFRICAN FIREFINCH',
 'AFRICAN FIREFINCH',
 'AFRICAN FIREFINCH',
 'AFRICAN FIREFINCH',
 'AFRICAN FIREFINCH',
 'AMERICAN AVOCET',
 'ALBATROSS',
 'ALBATROSS',
 'AMERICAN AVOCET',
 'ALBATROSS',
 'ALEXANDRINE PARAKEET',
 'ALEXANDRINE PARAKEET',
 'ALEXANDRINE PARAKEET',
 'ALEXANDRINE PARAKEET',
 'ALEXANDRINE PARAKEET',
 'AMERICAN AVOCET',
 'AFRICAN CROWNED CRANE',
 'AMERICAN AVOCET',
 'AMERICAN AVOCET',
 'AMERICAN AVOCET',
 'BALD EAGLE',
 'BALD EAGLE',
 'BALD EAGLE',
 'BALD EAGLE',
 'BALD EAGLE',
 'BALI STARLING',
 'BALI STARLING',
 'BALI STARLING',
 'BALI STARLING',
 'BALI STARLING',
 'BALTIMORE ORIOLE',
 'BALTIMORE ORIOLE',
 'BALTIMORE ORIOLE',
 'BALTIMORE ORIOLE',
 'BALTIMORE ORIOLE',
 'BANANAQUIT',
 'BANANAQUIT',
 'CANARY',
 'BANANAQUIT',
 'BANANAQUIT',
 'BANDED BROADBILL',
 'BANDED BROADBILL',
 'BANDED BROADBILL',
 'BANDED BROADBILL',
 'BANDED BROADBILL',
 'CAC

From the above result, I am confident to tell you that my model has achieved high accuracy in the sense that it reaches around 90% accuracy. However, there are one potential problem inside that it fails to accurately classify the ALBATROSS specy of bird. With a view to that, more data or more training is needed to improve our model.